In [1]:
import sys
sys.path.append('../..') 
from typing import Any
import numpy as np
import torch

from Deep import MyModel
from Deep.activation import ReLU
from Deep.activation.loss import MeanSquaredError, CrossEntropyLoss
from Deep.optimizer.gdo import GradientDescentOptimizer
import torchvision

import torchvision.transforms as transforms
from torchvision import datasets, transforms

import matplotlib.pyplot as plt

In [2]:
"""
多层感知机进行MINIST手写数据集的识别
"""
photo_nums = 64

transform = transforms.Compose([transforms.ToTensor()])

# 获取MNIST数据集
# windows
# train_dataset = datasets.MNIST(root='G:\学习文件\python学习\CODE\data', train=True, transform=transform, download=True)
# test_dataset = datasets.MNIST(root='G:\学习文件\python学习\CODE\data', train=False, transform=transform, download=True)
# linux
train_dataset = datasets.MNIST(root='/media/xc/学习/学习文件/python学习/CODE/data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='/media/xc/学习/学习文件/python学习/CODE/data', train=False, transform=transform, download=True)

# 定义数据加载器
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=photo_nums, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=photo_nums, shuffle=False)


In [3]:
loss_fn = CrossEntropyLoss()
layer_dict = {
    'first': {
        'type': 'linear',
        'in_features': 784,
        'out_features': 128,
        'activation': "ReLU"
    },
    'output': {
        'type': 'linear',
        'in_features': 128,
        'out_features': 10,
        'activation': "Softmax",
        'loss_fn': loss_fn
    }
}

op = GradientDescentOptimizer(lr=0.001, max_iterations=1)
model = MyModel(layers_dict=layer_dict, optimizer = op)

In [4]:
# 检查GPU是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "npu")
device

device(type='cuda')

In [8]:
epochs = 20
for epoch in range(epochs):
    k = 0
    for batch in train_loader:
        inputs, labels = batch

        x = inputs.numpy()
        y = labels.numpy()
        x = x.reshape(photo_nums, -1)
        x = np.asarray(x)
        y_one_hot = np.zeros((len(y), 10))
        for i in range(len(y)):
            y_one_hot[i, y[i]] = 1

        if x.shape[1] == 784:
            model.fit(x, y_one_hot)

        k += 1
        if k >= 500:
            break
    running_loss = model.loss[-1]
    print(f"{64*500} photos, Epoch {epoch + 1}, Loss: {running_loss}")

32000 photos, Epoch 1, Loss: 11.02604855800234
32000 photos, Epoch 2, Loss: 9.057483897798614
32000 photos, Epoch 3, Loss: 7.852928564279591
32000 photos, Epoch 4, Loss: 7.966068207295817
32000 photos, Epoch 5, Loss: 6.373314805551354
32000 photos, Epoch 6, Loss: 6.954159414352442
32000 photos, Epoch 7, Loss: 6.567632170514841
32000 photos, Epoch 8, Loss: 5.567408734736488
32000 photos, Epoch 9, Loss: 5.755600484518299
32000 photos, Epoch 10, Loss: 5.223381690628239
32000 photos, Epoch 11, Loss: 4.745601724214742
32000 photos, Epoch 12, Loss: 5.473827613939632
32000 photos, Epoch 13, Loss: 4.861826615018624
32000 photos, Epoch 14, Loss: 4.46836808266454
32000 photos, Epoch 15, Loss: 6.155879156280291
32000 photos, Epoch 16, Loss: 4.651920363950977
32000 photos, Epoch 17, Loss: 4.881135199640957
32000 photos, Epoch 18, Loss: 4.673845670106526
32000 photos, Epoch 19, Loss: 4.884347852641689
32000 photos, Epoch 20, Loss: 4.955103771458002


In [13]:
acc = 0.0
current = 0.0
loss = 0.0
n = 0
for batch in test_loader:
    inputs, labels = batch

    x = inputs.numpy()
    y = labels.numpy()
    x = np.asarray(x)
    y = np.asarray(y)
    x = x.reshape(photo_nums, -1)

    if x.shape[1] == 784:
        predict = model.predict(x)
        loss += model.loss[-1]
        n += 1
        max_indices = predict.argmax(axis=0)  # 沿着列的方向找到最大值的索引, 代表模型预测的分类
        acc = np.sum(max_indices == y) / predict.shape[1]
        current += acc

print(f"test_acc: {current/n}, test_loss: {loss/n}")

test_acc: 0.8727964743589743, test_loss: 4.955103771457999
